* Pulls pricing data from AWS APIs - ElastiCache, OpenSearch, Redshift, RDS
* Transform into hourly pricing data by SKU for RI analysis
* Export to csv

In [1]:
import pandas as pd

#### ElastiCache

## Reading API into CSV
dfdf = pd.read_csv('https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonElastiCache/current/index.csv', skiprows=5)

## Transforming data
dfdf = dfdf.fillna('')
dfdf = dfdf[dfdf['Unit'].isin(['Hrs', 'Quantity'])] # filter to just hourly and upfront fee costs
dfdf = dfdf[dfdf['PriceDescription'].str.contains('Outpost') == False] # remove pricing for outposts

## Creating SKU name field
cols = ['Region Code', 'Instance Type', 'Cache Engine']
dfdf['SKU_Name'] = dfdf[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)

## OnDemand Cost by SKU
df1 = dfdf[  (dfdf['TermType'] == 'OnDemand') & (dfdf['usageType'].str.contains('NodeUsage')) ]
df1 = df1.pivot(columns = ['PurchaseOption', 'LeaseContractLength'], index = 'SKU', values = 'PricePerUnit')
df1.columns = ['OnDemand']

## RI Upfront Cost by SKU
df2 = dfdf[ (dfdf['TermType'] == 'Reserved') & (dfdf['PriceDescription'] == 'Upfront Fee' ) ]
df2 = df2.pivot(columns = ['PurchaseOption', 'LeaseContractLength'], index = 'SKU', values = 'PricePerUnit')
df2.columns = ['UpfrontCost ' + '-'.join(col) for col in df2.columns.values]

## RI Hourly by SKU
df3 = dfdf[ (dfdf['TermType'] == 'Reserved') & (dfdf['PriceDescription'] != 'Upfront Fee') ]
df3 = df3.pivot(columns = ['LeaseContractLength', 'PurchaseOption'], index = 'SKU', values = 'PricePerUnit')
df3.columns = ['Hourly ' + '-'.join(col) for col in df3.columns.values]

## Merging into single table
dfec = pd.merge(df1, dfdf[['SKU', 'SKU_Name', 'Region Code', 'Location', 'serviceName', 'Instance Type' ]] ,how = "left", left_index = True, right_on = "SKU").drop_duplicates()
dfec = pd.merge(dfec, df2, how = "left", left_on = "SKU", right_index = True)
dfec = pd.merge(dfec, df3, how = "left", left_on = "SKU", right_index = True)

In [2]:
#### OpenSearch

## Reading API into CSV
dfdf = pd.read_csv('https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonES/current/index.csv', skiprows=5)
## Transforming data
dfdf = dfdf.fillna('')
dfdf = dfdf[dfdf['Unit'].isin(['Hrs', 'Quantity'])] # filter to just hourly and upfront fee costs

## Creating SKU name field
cols = ['Region Code', 'Instance Type']
dfdf['SKU_Name'] = dfdf[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)

## OnDemand Cost by SKU
df1 = dfdf[  (dfdf['TermType'] == 'OnDemand') & (dfdf['usageType'].str.contains('ESInstance')) ]
df1 = df1.pivot(columns = ['PurchaseOption', 'LeaseContractLength'], index = 'SKU', values = 'PricePerUnit')
df1.columns = ['OnDemand']

## RI Upfront Cost by SKU
df2 = dfdf[ (dfdf['TermType'] == 'Reserved') & (dfdf['PriceDescription'] == 'Upfront Fee' ) ]
df2 = df2.pivot(columns = ['PurchaseOption', 'LeaseContractLength'], index = 'SKU', values = 'PricePerUnit')
df2.columns = ['UpfrontCost ' + '-'.join(col) for col in df2.columns.values]

## RI Hourly by SKU
df3 = dfdf[ (dfdf['TermType'] == 'Reserved') & (dfdf['PriceDescription'] != 'Upfront Fee') ]
df3 = df3.pivot(columns = ['LeaseContractLength', 'PurchaseOption'], index = 'SKU', values = 'PricePerUnit')
df3.columns = ['Hourly ' + '-'.join(col) for col in df3.columns.values]

## Merging into single table
dfos = pd.merge(df1, dfdf[['SKU', 'SKU_Name', 'Region Code', 'Location', 'serviceName', 'Instance Type' ]] ,how = "left", left_index = True, right_on = "SKU").drop_duplicates()
dfos = pd.merge(dfos, df2, how = "left", left_on = "SKU", right_index = True)
dfos = pd.merge(dfos, df3, how = "left", left_on = "SKU", right_index = True)

In [3]:

#### RDS

## Reading API into CSV
dfdf = pd.read_csv('https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonRDS/current/index.csv', skiprows=5)
## Transforming data
dfdf = dfdf.fillna('')
dfdf = dfdf[dfdf['Unit'].isin(['Hrs', 'Quantity'])] # filter to just hourly and upfront fee costs
dfdf = dfdf[dfdf['PriceDescription'].str.contains('Outpost') == False] # remove pricing for outposts
dfdf = dfdf[dfdf['Location'] != 'Asia Pacific (Osaka-Local)'] #region code doesn't come through for this region - removing
dfdf = dfdf[dfdf['Storage'] != 'Aurora IO Optimization Mode'] ## remove pricing for IO Optimized - size flexibility auto adjusts to these
dfdf = dfdf[dfdf['Engine Media Type'] != 'Customer-provided']

## Creating SKU name field
cols = ['Region Code', 'Instance Type', 'Database Engine', 'Database Edition', 'License Model', 'Deployment Option']
dfdf['SKU_Name'] = dfdf[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)

## OnDemand Cost by SKU
df1 = dfdf[  (dfdf['TermType'] == 'OnDemand') & (dfdf['usageType'].str.contains('Usage')) ]
df1 = df1.pivot(columns = ['PurchaseOption', 'LeaseContractLength'], index = 'SKU', values = 'PricePerUnit')
df1.columns = ['OnDemand']

## RI Upfront Cost by SKU
df2 = dfdf[ (dfdf['TermType'] == 'Reserved') & (dfdf['PriceDescription'] == 'Upfront Fee' ) ]
df2 = df2.pivot(columns = ['PurchaseOption', 'LeaseContractLength'], index = 'SKU', values = 'PricePerUnit')
df2.columns = ['UpfrontCost ' + '-'.join(col) for col in df2.columns.values]

## RI Hourly by SKU
df3 = dfdf[ (dfdf['TermType'] == 'Reserved') & (dfdf['PriceDescription'] != 'Upfront Fee') ]
df3 = df3.pivot(columns = ['LeaseContractLength', 'PurchaseOption'], index = 'SKU', values = 'PricePerUnit')
df3.columns = ['Hourly ' + '-'.join(col) for col in df3.columns.values]

## Merging into single table
dfrds = pd.merge(df1, dfdf[['SKU', 'SKU_Name', 'Region Code', 'Location', 'serviceName', 'Instance Type' ]] ,how = "left", left_index = True, right_on = "SKU").drop_duplicates()
dfrds = pd.merge(dfrds, df2, how = "left", left_on = "SKU", right_index = True)
dfrds = pd.merge(dfrds, df3, how = "left", left_on = "SKU", right_index = True)

<ipython-input-3-a1a64e8e5f34>:4: DtypeWarning: Columns (11,48,50,56) have mixed types. Specify dtype option on import or set low_memory=False.
  dfdf = pd.read_csv('https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonRDS/current/index.csv', skiprows=5)


In [4]:
#### Redshift

## Reading API into CSV
dfdf = pd.read_csv('https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonRedshift/current/index.csv', skiprows=5)

## Transforming data
dfdf = dfdf.fillna('')
dfdf = dfdf[dfdf['Unit'].isin(['Hrs', 'Quantity'])] # filter to just hourly and upfront fee costs


## Creating SKU name field
cols = ['Region Code', 'Instance Type']
dfdf['SKU_Name'] = dfdf[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)

## OnDemand Cost by SKU
df1 = dfdf[  (dfdf['TermType'] == 'OnDemand') & (dfdf['usageType'].str.contains('Node')) ]
df1 = df1.pivot(columns = ['PurchaseOption', 'LeaseContractLength'], index = 'SKU', values = 'PricePerUnit')
df1.columns = ['OnDemand']

## RI Upfront Cost by SKU
df2 = dfdf[ (dfdf['TermType'] == 'Reserved') & (dfdf['PriceDescription'] == 'Upfront Fee' ) ]
df2 = df2.pivot(columns = ['PurchaseOption', 'LeaseContractLength'], index = 'SKU', values = 'PricePerUnit')
df2.columns = ['UpfrontCost ' + '-'.join(col) for col in df2.columns.values]

## RI Hourly by SKU
df3 = dfdf[ (dfdf['TermType'] == 'Reserved') & (dfdf['PriceDescription'] != 'Upfront Fee') ]
df3 = df3.pivot(columns = ['LeaseContractLength', 'PurchaseOption'], index = 'SKU', values = 'PricePerUnit')
df3.columns = ['Hourly ' + '-'.join(col) for col in df3.columns.values]

## Merging into single table
dfrs = pd.merge(df1, dfdf[['SKU', 'SKU_Name', 'Region Code', 'Location', 'serviceName', 'Instance Type' ]] ,how = "left", left_index = True, right_on = "SKU").drop_duplicates()
dfrs = pd.merge(dfrs, df2, how = "left", left_on = "SKU", right_index = True)
dfrs = pd.merge(dfrs, df3, how = "left", left_on = "SKU", right_index = True)

In [5]:
################## Create Final Table and Export

fdf = pd.concat([dfec, dfos, dfrds, dfrs])

## Filtering and Ordering Columns for Final Table
fdf = fdf[['OnDemand', 'SKU', 'SKU_Name', 'Region Code', 'Location', 'serviceName', 'Instance Type',
          'Hourly 1yr-No Upfront', 'Hourly 1yr-Partial Upfront', 'Hourly 1yr-All Upfront',
          'UpfrontCost Partial Upfront-1yr', 'UpfrontCost All Upfront-1yr',
          'Hourly 3yr-No Upfront', 'Hourly 3yr-Partial Upfront', 'Hourly 3yr-All Upfront',
          'UpfrontCost Partial Upfront-1yr', 'UpfrontCost All Upfront-1yr']]

## Export
fdf.to_csv('Non-EC2-RI-pricing.csv', index = False)

In [ ]:
fdf.columns

Index(['OnDemand', 'SKU', 'SKU_Name', 'Region Code', 'Location', 'serviceName',
       'Instance Type', 'UpfrontCost Partial Upfront-3yr',
       'UpfrontCost All Upfront-1yr', 'UpfrontCost Partial Upfront-1yr',
       'UpfrontCost All Upfront-3yr', 'UpfrontCost Heavy Utilization-3yr',
       'UpfrontCost Heavy Utilization-1yr', 'Hourly 3yr-Partial Upfront',
       'Hourly 1yr-No Upfront', 'Hourly 1yr-All Upfront',
       'Hourly 3yr-No Upfront', 'Hourly 1yr-Partial Upfront',
       'Hourly 3yr-All Upfront', 'Hourly 3yr-Heavy Utilization',
       'Hourly 1yr-Heavy Utilization'],
      dtype='object')